In [3]:
# --- Forward Chaining with Unification ---

import re
from copy import deepcopy

# Helper functions
def parse_predicate(pred):
    """Parse predicate like P(x,y) -> ('P', ['x','y'])"""
    name, args = pred.split('(')
    args = args[:-1].split(',')
    return name.strip(), [a.strip() for a in args]

def substitute(predicate, subs):
    """Apply substitutions to a predicate"""
    name, args = parse_predicate(predicate)
    new_args = [subs.get(a, a) for a in args]
    return f"{name}({', '.join(new_args)})"

def unify(x, y, subs=None):
    """Simple unification algorithm"""
    if subs is None:
        subs = {}
    if x == y:
        return subs
    if re.match(r"^[a-z]\w*$", x):  # variable
        subs[x] = y
        return subs
    if re.match(r"^[a-z]\w*$", y):  # variable
        subs[y] = x
        return subs
    if '(' in x and '(' in y:
        nx, argsx = parse_predicate(x)
        ny, argsy = parse_predicate(y)
        if nx != ny or len(argsx) != len(argsy):
            return None
        for a, b in zip(argsx, argsy):
            subs = unify(a, b, subs)
            if subs is None:
                return None
        return subs
    return None

# Knowledge base
facts = {
    "American(Robert)",
    "Missile(M1)",
    "Owns(Robert, M1)",
    "Enemy(Nono, America)"
}

rules = [
    (["Missile(x)"], "Weapon(x)"),
    (["American(x)", "Weapon(y)", "Sells(x, y, z)", "Hostile(z)"], "Criminal(x)"),
    (["Enemy(x, America)"], "Hostile(x)")
]

# Additional fact: Robert sells the missile to Nono
facts.add("Sells(Robert, M1, Nono)")

def forward_chain(facts, rules):
    inferred = set()
    while True:
        new_facts = set()
        for premises, conclusion in rules:
            # Try to find substitutions that satisfy premises
            possible_subs = [{}]
            for premise in premises:
                new_subs = []
                for fact in facts:
                    for subs in possible_subs:
                        prem = substitute(premise, subs)
                        s = unify(prem, fact, deepcopy(subs))
                        if s is not None:
                            new_subs.append(s)
                possible_subs = new_subs

            for subs in possible_subs:
                new_fact = substitute(conclusion, subs)
                if new_fact not in facts:
                    new_facts.add(new_fact)

        if not new_facts:
            break
        facts |= new_facts
        inferred |= new_facts
    return inferred

# Run inference
inferred = forward_chain(facts, rules)

print("All known facts after inference:")
for f in sorted(facts):
    print("  ", f)

if "Criminal(Robert)" in facts:
    print("\n✅ Proven: Robert is a Criminal.")
else:
    print("\n❌ Could not prove Criminal(Robert).")


All known facts after inference:
   American(Robert)
   Criminal(Robert)
   Enemy(Nono, America)
   Hostile(Nono)
   Missile(M1)
   Owns(Robert, M1)
   Sells(Robert, M1, Nono)
   Weapon(M1)

✅ Proven: Robert is a Criminal.
